# Connectivity Inference V2

This notebook applied the V2 of the connectivity inference on the adEx implementation of Taehoon of "Susin, Eduarda, and Alain Destexhe. 2021."

In [4]:
############################
######## SETUP  ###########

#####  General Imports ######
import numpy as np
from brian2 import *
from matplotlib import pyplot
import sys
import networkx

##### setup interactive ####
%matplotlib notebook 

In [6]:
#### Simulation Scripts #####
sys.path.append('../simulations/')
from wp2_adex_model_script import * 

###### Utility Scripts #####
sys.path.append('../tools')
#import wp2_util as  wp2_util

In [8]:
###########################
####### RUN SIMULATION ####

### choose parameters ###
params = dict()
params['sim_time'] = float(30) # simulation time in seconds 
params['a'] = float( 4*10**(-9))    # subthreshold adaption constant [Siemens]
params['b'] = float(0.0805 * 10**(-9)) # spike-triggert adaption constant [A ]
params['N'] = int(10) #  no of neurons
params['ge']=float(4*10**(-9)) # excitatory synaptic conductance [S] - [VALUE ???]
params['gi']=float(4*10**(-9)) # inhibitory synaptic conductance [S] - [VALUE ???]

root_dir = 'simData'

curr_dir = root_dir + '/' + str(params['N'])

#if(not(os.path.exists(curr_dir))):
#    os.makedirs(curr_dir)

params['save_fol'] = curr_dir


# run simulation
#trace, spikes, S = simpleNetV2(n, p, c, t, ws, tauw, a, b, Vr) # Regular spiking 
result = run_sim(params)

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.03s, trying other methods took 0.22s). [brian2.stateupdaters.base.method_choice]
INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.01s, trying other methods took 0.13s). [brian2.stateupdaters.base.method_choice]


The time difference is : 115.25729609599989
[]
[ 1.5 13.7 17.7 22.4 28.6 34.6 38.6 39.8 42.6 47.9 53.5 54.4 55.8 70.7
 78.8 83.  83.1 83.6 87.6 87.8 89.2 89.8 93.2]
simulation successfullly ran for 10_4e-09_8.05e-11_30.0_4e-09_4e-09
